<a href="https://colab.research.google.com/github/pravin691983/insaid-capstone_project_1/blob/development/SourceCode/insaid_capstone_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INSAID - Capstone Project on Data Analytics**


# Problem Statement

**InsaidTelecom**, one of the leading telecom players, understands that customizing offering
is very important for its business to stay competitive.
Currently, InsaidTelecom is seeking to leverage behavioral data from more than 60% of the 50 million mobile devices active daily in India
to help its clients better understand and interact with their audiences.



# Objective

In this **consulting assignment**, Insaidians are expected to build a dashboard to understand
user's demographic characteristics based on their mobile usage, geolocation, and mobile device properties.
Doing so will help millions of developers and brand advertisers around the world pursue
data-driven marketing efforts which are relevant to their users and catered to their preferences.

To help the customer the consultants are expected to have depth of clarity in the underlying data.
How much effort has been put into cleansing and purifying the data will decide how closely have you looked at the data.
How detailed is the observation stated in the submission report and finally how well a group presents their consulting journey.

Please remember this is an analytics consulting hence, your efforts in terms of
finding user behavior is going to directly impact the company's offerings.
Do help the company understand what is the
right way forward and suggest actionable insights from marketing and product terms.

# Data Loading and Description

// TODO: Add Description



***Importing a library that is not in Colaboratory*** 
<Br>
To import a library that's not in Colaboratory by default, you can use !pip install or !apt-get install.

In [0]:
!pip install -q mysql-connector-python-rf

In [0]:
#Import Required packages
import mysql.connector
from mysql.connector import Error
import pandas as pd

# Data Profiling

// TODO: 

# Questions

// TODO: Add Question List

# Conclusion

// TODO :